# Rating of a Single-Stage Vapor Compression Machine
---

**Importing libraries and notebook preparation**

In [1]:
from pathlib import Path

from hvac import Quantity
from hvac.fluids import Fluid, HumidAir, CoolPropWarning
from hvac.vapor_compression import VariableSpeedCompressor
from hvac.vapor_compression.machine import SingleStageVaporCompressionMachine
from hvac.heat_exchanger.fintube.continuous_fin import PlainFinTubeCounterFlowAirEvaporator
from hvac.heat_exchanger.fintube.continuous_fin import PlainFinTubeCounterFlowAirCondenser

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=CoolPropWarning)

Q_ = Quantity

## Configuration of the Vapor Compression Machine

A single-stage vapor compression machine is composed of a compressor, an evaporator, a condenser, and an expansion device. Air at a given state and mass flow rate enters the evaporator. Also at the condenser, air enters at a given state and mass flow rate. The expansion device regulates the mass flow rate of refrigerant in order to maintain the set degree of refrigerant superheating. The compressor (either with fixed or variable speed drive) runs at a given compressor speed. The question is what the steady-state performance of the vapor compression machine will be under the given operating conditions.

To configure a single-stage vapor compression machine, we first need to define compressor, evaporator and condenser.

**Compressor**<br>
After rating the evaporator (see notebook *ex02_air_evaporator.ipynb*), we re-selected a variable speed compressor DANFOSS VTZ054-G for refrigerant R134a using the selection software again from the compressor manufacturer (see notebook *ex03_air_condenser.ipynb*). The polynomial coefficients of the functions describing the performance quantities of this compressor were saved in a csv-file.   

First, define the refrigerant:

In [2]:
R134a = Fluid('R134a')

Now we can define the compressor:

In [3]:
compressor = VariableSpeedCompressor(
    coeff_file=Path("compressor_data/VTZ054-G_R134a.csv"),
    refrigerant_type=R134a,
    units={'m_dot': 'kg / hr', 'speed': '1 / s'}
)

**Evaporator**<br>
We will use the same air evaporator defined in notebook no. 2 (plain fin-tube counterflow evaporator).

In [4]:
evaporator = PlainFinTubeCounterFlowAirEvaporator(
    W_fro=Q_(0.731, 'm'),
    H_fro=Q_(0.244, 'm'),
    N_rows=3,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm')
)

**Condenser**<br>
We will use the same air condenser defined in notebook no. 3 (plain fin-tube counterflow condenser).

In [5]:
condenser = PlainFinTubeCounterFlowAirCondenser(
    W_fro=Q_(1.003, 'm'),
    H_fro=Q_(0.334, 'm'),
    N_rows=5,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm')
)

**Vapor compression machine**<br>
The creation of the vapor compression machine is straightforward. First, you pass the evaporator, condenser, and compressor model to the constructor. Then, you specify the refrigerant and the degree of refrigerant superheating set on the expansion device:

In [6]:
machine = SingleStageVaporCompressionMachine(
    evaporator, condenser, compressor,
    R134a, dT_sh=Q_(10, 'K')
)

## Specify Known Operating Conditions and Determine Steady-State Machine Operation

The following inputs will unambiguously determine the operation of the vapor compression machine:
- the mass flow rate of air through the evaporator
- the mass flow rate of air through the condenser
- the air state at the evaporator inlet
- the air state at the condenser inlet
- the compressor speed (only in case of a variable speed compressor)

To determine the steady-state performance of the machine, an iterative solving technique is used inspired by the method explained in Chapter 14 of the book *Stoecker, W. F., & Jones, J. W. (1982). Refrigeration and air conditioning. McGraw-Hill International Editions, Mechanical Technology Series.* 
The implemented rating routine uses a minimization algorithm from scipy (`scipy.optimize.minimize`) to determine the evaporation and condensation temperature for which the deviation is minimal between the refrigerant mass flow rate according to the compressor model, and the refrigerant mass flow rate according to the evaporator model (i.e., the mass flow rate let through by the expansion device to maintain the set degree of refrigerant superheating at the evaporator's outlet). The routine needs initial guesses for the evaporation and condensing temperature to start. If the user does not provide these initial guesses, the rating routine will guess the initial values by itself. However, it may happen that no solution can be found with one set of initial guesses, but that a solution is found with another set.

To determine the steady-state performance of the vapor compression machine in this example, we will use the operating conditions we have used so far. The air mass flow rate through the condenser is taken from notebook no. 3 (*ex03_air_condenser.ipynb*). The compressor speed is the speed taken from the compressor selection program when we rated the air condenser.

In [7]:
output = machine.rate(
    evp_air_in=HumidAir(Tdb=Q_(24.0, 'degC'), RH=Q_(50, 'pct')),
    evp_air_m_dot=Q_(1500, 'kg / hr'),
    cnd_air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    cnd_air_m_dot=Q_(3216.315, 'kg / hr'),
    n_cmp=Q_(4343, '1 / min'),
    T_evp_ini=Q_(5, 'degC'),
    T_cnd_ini=Q_(50, 'degC')
)

[23264 | hvac.vapor_compression.machine | INFO] Starting machine operation analysis with...
[23264 | hvac.vapor_compression.machine | INFO] n_cmp = 4343 1/min
[23264 | hvac.vapor_compression.machine | INFO] evp_air_in: 24.00 °C, 50 %
[23264 | hvac.vapor_compression.machine | INFO] evp_air_m_dot: 1500.00 kg/h
[23264 | hvac.vapor_compression.machine | INFO] cnd_air_in: 35.00 °C, 30 %
[23264 | hvac.vapor_compression.machine | INFO] cnd_air_m_dot: 3216.32 kg/h
[23264 | hvac.vapor_compression.machine | INFO] Iteration 1: Try with: T_evp = 5.000 °C, T_cnd = 50.000 °C
[23264 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate from compressor = 169.615 kg/h
[23264 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering condenser with T = 86.440 °C, P = 13.179 bar
[23264 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving condenser with T = 45.490 °C, P = 13.179 bar
[23264 | hvac.vapor_compression.machine | INFO] Iteration 1:

In [8]:
print(output.to_text())

evp_air_m_dot = 1500.000 kg/h
cnd_air_m_dot = 3216.315 kg/h
evp_air_in = 24.000 °C DB, 9.340 g/kg AH (50 % RH)
cnd_air_in = 35.000 °C DB, 10.590 g/kg AH (30 % RH)
n_cmp = 4343.000 1/min
dT_sh = 10.000 K
evp_air_out = 13.020 °C DB, 7.239 g/kg AH (77 % RH)
cnd_air_out = 45.295 °C DB, 10.590 g/kg AH (17 % RH)
evp_Q_dot = 6.891 kW
cnd_Q_dot = 9.442 kW
cmp_W_dot = 2.553 kW
rfg_m_dot = 169.313 kg/h
COP = 3.699 frac
EER = 2.699 frac
evp_eps = 0.5468 frac
cnd_eps = 0.5580 frac
T_evp = 4.972 °C
P_evp = 3.493 bar
T_cnd = 50.107 °C
P_cnd = 13.215 bar
dT_sc = 4.965 K
suction_gas = 14.972 °C, 410.599 kJ/kg, 3.493 bar
discharge_gas = 86.583 °C, 464.883 kJ/kg, 13.215 bar
liquid = 45.142 °C, 264.113 kJ/kg, 13.215 bar
mixture = 4.972 °C, 264.113 kJ/kg, 3.493 bar, 29 %
evp_air_dP = 22.112 Pa
cnd_air_dP = 53.514 Pa

